In [8]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")


# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust tissue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("complete_data/data.nii.gz").get_data()

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

In [14]:
dti_fa.affine

array([[-2.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         9.66024094e+01],
       [ 0.00000000e+00,  1.99708164e+00, -1.08000085e-01,
        -1.16507339e+02],
       [ 0.00000000e+00,  1.08004414e-01,  1.99700165e+00,
        -4.14616928e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [5]:
model = ShoreModel(gtab)
fit = model.fit(data, wm_mask, gm_mask, csf_mask, dti_mask, 
                    dti_fa, dti_vecs, verbose=True)

100%|██████████| 460800/460800 [00:32<00:00, 14189.76it/s] 


In [6]:
fit.old_save('/home/olivier/Devel/test/samuel/in4michi/bonndit_result/')
fit.save("/home/olivier/Devel/test/samuel/in4michi/bonndit_result/"+"fit.pkl")

In [3]:
fit = ShoreFit.load("/home/olivier/Devel/test/samuel/in4michi/bonndit_result/"+"fit.pkl")

In [5]:
out, wmout, gmout, csfout, mask, meta = fit.fodf("data.nii", verbose=True)

Optimization:  33%|███▎      | 9/27 [00:00<00:00, 79.95it/s]

Condition number of M: inf


Optimization: 100%|██████████| 27/27 [00:00<00:00, 68.32it/s]


In [8]:
img = nib.Nifti1Image(out, data.affine)
nib.save(img, '/home/olivier/Devel/test/samuel/in4michi/bonndit_result/odf.nii.gz')

In [14]:
from bonndit.shore import gtab_reorient, gtab_rotate
from dipy.core.gradients import GradientTable
import numpy as np
GRADIENTS = np.array([[2500,0,0],
                      [0,2500,0],
                      [0,0,2500],
                      [0,0,0]])
gtab = GradientTable(GRADIENTS)
old_vec = np.array((0, 0, 1))
new_vec = np.array((1, 0, 0))
gtab_reorient(gtab, old_vec, new_vec).gradients

array([[    0.,     0., -2500.],
       [    0.,  2500.,     0.],
       [ 2500.,     0.,     0.],
       [    0.,     0.,     0.]])